In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import PyPDF2
import os


#### Collect the links on the page

In [2]:
url = "https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/"
response = requests.get(url + "?showSvar=true&term=&page=1&isFilterOpe")

html_doc = response.text

soup = bs(html_doc, 'html.parser')

# Find the ul class that contains the articles
articles = soup.find("ul", class_="link-list")

# Create a dictionary to store the sites and pdfs
sites = {}
pdfs = {}

# Iterate through every li tag in the ul class
for article in articles.find_all("li"):
    if article.text.find("PDF") != -1:
        # Removes the (PDF) from the name of the actor (the writer of the article)
        actor = article.text.split("(")[0][:-1]
        pdfs[actor] = "https://www.regjeringen.no/" + article.find("a")["href"]
        continue
    sites[article.text] = url + article.find("a")["href"]


print(sites)
print(pdfs)


{'Abelia ': 'https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/?uid=c0ad43af-ee84-40ce-bcbe-9288ae5abe33', 'Advokatforeningen ': 'https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/?uid=3dd245ce-31b6-4975-bcc2-702b9da087f8', 'Agder fylkeskommune ': 'https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/?uid=ed6abd47-ef58-4153-974b-5d31b441aa98', 'Akademikerne  ': 'https://www.regjeringen.no/no/dokumenter/horing-nou-2023-25-omstilling-til-lavutslipp-veivalg-for-klimapolitikken-mot-2050-rapport-av-klimautvalget-2050/id3009052/?uid=4b024a54-f322-462d-ac59-8081f66772ab', 'Alliansen ny landbrukspolitikk ': 'https://www.regjeringen.no/no/dokumenter/horing-nou-2023

#### Download pdfs

In [3]:
# Make a directory to store the pdfs
output_dir = "../../dataset/pdf"

os.makedirs(output_dir, exist_ok=True)

# Iterate through the pdfs and download them
for actor, pdf in pdfs.items():
    response = requests.get(pdf)
    output_file = os.path.join(output_dir, actor + ".pdf")
    with open(output_file, "wb") as f:
        f.write(response.content)
        

#### Read the pdfs and store sections to a csv file

In [4]:

# Preprocessing the pdfs, removing the metadata from the text, and saving the date to the dictionary

def preprocess_pdf(text):
    # Extract the date from the text
    date_index = text.find("Dato")
    date = text[date_index:date_index+20].split("\n")[0].split(" ")[1]

    # Find the beginning of the text
    start_index = text.find("Høringsinnspill")
    text = text[start_index:]
    
    # Split into sections
    sections = text.split("  \n")

    for i in range(len(sections)):
        # Combine the section with the previous one or the next one if it is too short
        """if len(sections[i]) < 100:
            # if the previous section is longer than the next one
            if i+1 < len(sections):

                sections[i] = sections[i] + sections[i+1]
                i += 1
                """
        print("Index", i)
        print(sections[i])

# Iterate through the pdfs and extract the text
        
texts = {}

for actor, pdf in pdfs.items():

    pdf_file = os.path.join(output_dir, actor + ".pdf")
    pdf_obj = open(pdf_file, "rb")
    pdf_reader = PyPDF2.PdfReader(pdf_obj)
    num_pages = len(pdf_reader.pages)
    text = ""
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    preprocess_pdf(text)
    #print(text.__repr__())
    texts[actor] = text
    break

#texts["Akershus fylkeskommune"]

Index 0
Høringsinnspill til Klimautvalget 2050 
Akershus fylkeskommune viser til høring av Klimautvalgets NOU 2023: 25 Omstilling til 
lavutslipp – Veivalg for klimapolitikken mot 2050. Høringsuttalelsen er gitt på vegne av 
Akershus fylkeskommune med forbehold om politisk vedtak.
Index 1
Akershus fylkeskommune skal bidra til at Norge oppfyller sine internasjonale forpliktelser i 
Naturavtalen, Parisavtalen og klimaavtalen med EU fra 2019. NOU 2023:25 tillegger 
fylkeskommuner og kommuner en sentral rolle i utviklingen mot lavutslippssamfunnet 2050. 
Klimautvalget presiserer at Norge aldri vil nå målene for lavutslippssamfunnet 2050 om vi ikke 
lykkes med omstilling i kommunene.
Index 2
Arbeidet til Klimautvalget er nytenkende og effektfullt. Akershus fylkeskommune vil spesielt 
trekke frem utvalgets tiltaks-pyramide unngå, flytte og forbedre (UFF) som et tydelig 
rammeverk for prioritering av strategier og tiltak. Dette vil bidra til at flere trekker i samme 
retning i praktisk klimaa

In [63]:
df_texts = pd.DataFrame(columns=['actor', 'paragraph', 'date'])

# Read the html files and extract the text

def preprocess_html(texts : list):
    # Remove the html tags
    paragraphs = [paragraph.text.strip() for paragraph in texts if paragraph.text != ""]

    # Remove the header "Høringssvar til NOU 2023: ...."
    paragraphs = [sections for sections in paragraphs if sections.find("Høringssvar til NOU 2023") == -1]

    return paragraphs




for actor, site in sites.items():
    response = requests.get(site)
    html_doc = response.text
    soup = bs(html_doc, 'html.parser')
    # Find the text of the article
    texts = soup.find("div", class_="article-body").find_all("p")
    text = preprocess_html(texts)

    # Find the date of the article
    date = soup.get_text().split("Dato")[1].split("\n")[0].split(" ")[1]


    for paragraph in text:
        df_texts.loc[len(df_texts)] = [actor, paragraph, date]
    


print(actor)
df_texts


KeyboardInterrupt: 

In [62]:
df_texts

,actor,text,date
0,Abelia,Abelia er foreningen for kunnskaps- og teknolo...,30.01.2024
1,Abelia,Innledningsvis vil vi takke for det grundige o...,30.01.2024
2,Abelia,Vi er enig med utvalget i at klimapolitikken m...,30.01.2024
3,Abelia,Over de siste åtte årene har Abelia målt norsk...,30.01.2024
4,Abelia,Vi vil gjerne oppfordre til at det blir satt s...,30.01.2024
...,...,...,...
547,Buskerud fylkeskommune,Klimautvalgets anbefalinger vil pålegge kommun...,30.01.2024
548,Buskerud fylkeskommune,Lavutslippssamfunnet kan kun skapes gjennom go...,30.01.2024
549,Buskerud fylkeskommune,Buskerud fylkeskommune understreker viktighete...,30.01.2024
550,Buskerud fylkeskommune,Buskerud fylkeskommune understreker at det vil...,30.01.2024
